In [44]:
import pandas as pd
from sqlalchemy import create_engine, text,insert, MetaData, Table, Column, VARCHAR, INT
from sqlalchemy.exc import InvalidRequestError
import sqlite3
import os
from dotenv import load_dotenv

from etl.extract.extract import extract_api
from etl.transform.transform import clean_earthquake_data
from config.db_config import load_db_config

## Loading Data Into In-Memory Database

In [2]:
teststarttime = '2025-04-05T00:00:46'
testendtime = '2025-05-05T17:05:58'
dfa = extract_api(teststarttime, testendtime)

In [3]:
dfa = clean_earthquake_data(dfa)

In [ ]:
dfa.head()

In [9]:


data = {
    'customer_id': range(1, 10001),  # 10,000 rows
    'name': [f'Customer {i}' for i in range(1, 10001)],
    'email': [f'customer{i}@example.com' for i in range(1, 10001)],
    'city': ['City A' if i % 2 == 0 else 'City B' for i in range(1, 10001)]
}


df = pd.DataFrame(data)
     


In [ ]:
engine = create_engine('sqlite:///:memory:')

with engine.connect() as conn:
    conn.execute(text('''
        CREATE TABLE earthquakes (
            id VARCHAR(20) PRIMARY KEY,
            magnitude INT,
            location VARCHAR(255),
            time DATETIME,
            type VARCHAR(20),
            longitude INT,
            latitude INT,
            depth INT,
            nearestCity VARCHAR(70)                        
        );
    '''))

dfa.to_sql('earthquakes', engine, if_exists='append', index=False, method='multi', chunksize=1000)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text('SELECT COUNT(*) FROM earthquakes'))
    print(f'Total rows imported: {result.fetchone()[0]}')

    # Optionally, display a few rows to verify
    result = conn.execute(text('SELECT * FROM earthquakes LIMIT 5'))
    for row in result:
        print(row)

## Loading Data Into Local Postgres Database

In [ ]:

db_details = load_db_config()['target_database']

In [ ]:
try:
    engine = create_engine(f'postgresql://{db_details['user']}:{db_details['password']}@{db_details['host']}:{db_details['port']}/{db_details['dbname']}')
except Exception as e:
    print(f'Cannot connect: {e}')

In [34]:
meta = MetaData()

In [ ]:

try:
    earthquake_test_1 = Table('earthquake_test_1',meta, Column('id',INT ,primary_key=True))
    meta.create_all(engine)
except InvalidRequestError as e: 
    print(f'Table already exists: {e}')                      



In [ ]:
dfa[:10].to_sql('earthquakes_test_2',engine,if_exists='replace')

In [ ]:
conn = engine.connect()
output = conn.execute("SELECT * FROM earthquakes_test_2")
print(output.fetchall())